In [ ]:
# https://docs.google.com/spreadsheets/d/1OZQjI2cVwLeYBkJAA3V1pmr8k1wnM6vrFtlOjTq7Kgo/edit?usp=drivesdk

# 1. kbo 홈페이지에서 나오는 승률 https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx
# 2. kbo 홈페이지 > 기록•순위 > 관중 현황 > 일자별 관중 현황에서 구단 이름별로 관중수 기록하기 (ex. 아래 사진에서 2025/04/11 NC 롯데 13728은 NC 관중수, 롯데 관중수에 둘 다 13728로 기록함) https://www.koreabaseball.com/Record/Crowd/GraphDaily.aspx

#     ![1000070933.jpg](attachment:cc8d1a90-7811-4176-acf0-a1162d3a1ba9:1000070933.jpg)

# 3. 유튜브 TVING SPORTS에 올라오는 경기 하이라이트 조회수를 오후 6시를 기준으로 2와 같이 기록함 https://youtube.com/@tvingsports?si=1Lyj8nLcyGIicNt2
# 4. 인스타그램 각 구단별로 경기 결과가 올라오는 게시물, 그게 없다면 경기 이후 올라온 첫번째 게시물을 오후 6시를 기준으로 각각 기록함

# KIA https://www.instagram.com/always_kia_tigers?igsh=NGJsMWljZGw4MDE5

# 삼성 https://www.instagram.com/samsunglions_baseballclub?igsh=dHZhbG5rbjQ3N3Q1

# LG https://www.instagram.com/lgtwinsbaseballclub?igsh=MTBncnAzNTNqczVsYw==

# 두산 https://www.instagram.com/doosanbears.1982?igsh=MXFkaW44NDdrbHVkOQ==

# KT https://www.instagram.com/ktwiz.pr?igsh=MTVyNXk5c29ldzJxbg==

# SSG https://www.instagram.com/ssglanders.incheon?igsh=YTJqcGZlMDRmMWl6

# 롯데 https://www.instagram.com/busanlottegiants?igsh=NDZzM2YxdWp2dmU5

# 한화 https://www.instagram.com/hanwhaeagles_soori?igsh=MWpqaGRyc2d0YzVybw==

# NC https://www.instagram.com/ncdinos2011?igsh=MThxaGwxeWhhaDRydw==

# 키움 https://www.instagram.com/heroesbaseballclub?igsh=eHZwMGZsajk2bTE1

In [ ]:
# 항목 이름	설명	예시
# 요소 복사 (Copy element)	선택한 HTML 요소 전체를 복사 (outerHTML과 같음)	<td>0.87</td>
# outerHTML 복사	요소 전체의 HTML을 복사 (자기 자신 포함)	<td style="...">0.87</td>
# selector 복사	해당 요소를 선택할 수 있는 CSS 셀렉터를 복사	#tableId > tbody > tr:nth-child(2) > td:nth-child(2)
# JS 경로 복사 (Copy JS path)	JS에서 document.querySelector()로 사용할 수 있는 경로	document.querySelector("#tableId > tbody > tr:nth-child(2) > td:nth-child(2)")
# 스타일 복사 (Copy styles)	요소에 적용된 CSS 스타일만 복사 (인라인 및 계산된 스타일 포함 가능)	color: red; font-size: 14px;
# XPath 복사	XPath 표현식으로 요소를 식별할 수 있는 경로 복사	//*[@id="tableId"]/tbody/tr[2]/td[2]
# 전체 XPath 복사 (Full XPath)	문서 전체를 기준으로 절대 XPath를 복사	/html/body/div[2]/table/tbody/tr[2]/td[2]

# 일자 승률, 일자 관중, 매일 조회수 현황


In [ ]:
# TODO: 함수화 필요
# TODO: 자동 저장 필요
# TODO: 매일 함수화 필요

In [87]:
import os
import re
from datetime import datetime, timedelta, timezone

import requests
from dotenv import load_dotenv

# 환경변수 불러오기
load_dotenv()
API_KEY = os.getenv("YOUTUBE_KEY")

# TVINGSPORTS 채널 ID
CHANNEL_ID = "UC8JtQf77wqhVpOQ8Cze8JjA"

# 어제 날짜 (KST 기준, 예: '4/12')
KST = timezone(timedelta(hours=9))
yesterday_kst = datetime.now(KST).date() - timedelta(days=1)
yesterday_str = f"{yesterday_kst.month}/{yesterday_kst.day}"


# 영상 ID 수집
def get_video_ids(channel_id, max_results=20):
    search_url = "https://www.googleapis.com/youtube/v3/search"
    params = {
        "key": API_KEY,
        "channelId": channel_id,
        "part": "snippet",
        "order": "date",
        "maxResults": max_results,
        "type": "video",
    }
    response = requests.get(search_url, params=params)
    response.raise_for_status()
    items = response.json().get("items", [])
    return [item["id"]["videoId"] for item in items]


# 영상 제목 + 조회수 수집
def get_video_details(video_ids, date_keyword):
    details_url = "https://www.googleapis.com/youtube/v3/videos"
    params = {"key": API_KEY, "part": "snippet,statistics", "id": ",".join(video_ids)}
    response = requests.get(details_url, params=params)
    response.raise_for_status()
    items = response.json().get("items", [])

    results = []
    for item in items:
        title = item["snippet"]["title"]
        views = int(item["statistics"].get("viewCount", 0))
        
        # 🎯 날짜 포함 + 'KBO 리그' 포함하는 영상만
        if date_keyword in title and 'KBO 리그' in title:
            results.append({"title": title, "views": views})
    return results


def get_view_data(filtered_videos):
    view_data = {}
    for video in filtered_videos:
        title = video["title"]
        views = video["views"]

        match = re.search(r"\[(.*?) vs (.*?)\]", title)
        if match:
            team1, team2 = match.group(1), match.group(2)
            view_data[f"{team1}"] = views
            view_data[f"{team2}"] = views
            
    return view_data

# 실행
if __name__ == "__main__":
    try:
        video_ids = get_video_ids(CHANNEL_ID)
        filtered_videos = get_video_details(video_ids, yesterday_str)
        view_data = get_view_data(filtered_videos)
        print(view_data)
        
    except Exception as e:
        print("⚠️ 오류 발생:", e)

{'롯데': 152961, 'NC': 152961, '키움': 168282, '한화': 168282, '두산': 168297, 'LG': 168297}


In [88]:
from datetime import datetime, timedelta, timezone

import requests
from bs4 import BeautifulSoup

# 어제 날짜 KST 기준
KST = timezone(timedelta(hours=9))
yesterday = datetime.now(KST).date() - timedelta(days=1)
yesterday_str = yesterday.strftime("%m/%d")  # 예: '04/12'


# 관중 수 테이블에서 어제 날짜 기준 데이터 가져오기
def get_crowd_data():
    url = "https://www.koreabaseball.com/Record/Crowd/GraphDaily.aspx"
    response = requests.get(url)
    response.encoding = "utf-8"

    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.table.find_all("tr")

    crowd_data = {}

    for row in rows[1:]:  # 첫 번째 행은 헤더
        cols = row.find_all("td")
        date = cols[0].text.strip()

        try:
            KST = timezone(timedelta(hours=9))
            yesterday_kst = datetime.now(KST).date() - timedelta(days=1)
            yesterday_str = yesterday_kst.strftime("%Y/%m/%d")
            if date == yesterday_str:
                # 예: '[두산-LG]' → 두산, LG
                home_team_name = cols[2].text.strip()
                out_team_name = cols[2].text.strip()
                crowd_number = cols[5].text.strip()

                crowd_data[f"{out_team_name}"] = crowd_number
                crowd_data[f"{home_team_name}"] = crowd_number

        except Exception as e:
            print(f"날짜 파싱 오류: {e}")

    return crowd_data


get_crowd_data()

{'NC': '15,222', 'LG': '22,938', '한화': '15,457'}

In [77]:
import requests
from bs4 import BeautifulSoup


def get_team_winrate_data():
    url = "https://www.koreabaseball.com/Record/TeamRank/TeamRankDaily.aspx"
    response = requests.get(url)
    response.encoding = "utf-8"

    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.table.find_all("tr")

    team_data = {}

    for row in rows[1:]:  # 첫 번째 행은 헤더
        cols = row.find_all("td")
        if len(cols) >= 7:
            team_name = cols[1].text.strip()
            win_rate = cols[6].text.strip()
            team_data[team_name] = win_rate

    return team_data


get_team_winrate_data()

{'LG': '0.875',
 'SSG': '0.643',
 '삼성': '0.588',
 'KT': '0.533',
 '롯데': '0.471',
 'NC': '0.400',
 '두산': '0.389',
 '한화': '0.389',
 '키움': '0.389',
 'KIA': '0.375'}